# QProgressBar

A progress bar widget notifies the users of the progress of an operation and reassures them that the program is still running.

To create a progress bar widget, we can use the QProgressBar class. It has three important values:
- The minimum value.
- The maximum value.
- The current step value.

By default, the `minimum value defaults to zero`, and the `maximum value defaults to 100`. To update the progress, you `increase the current step value`. By doing this, the progress bar will display the percentage of steps that have been completed.

The QProgressBar class uses the following formula to calculate the progress of the steps:

```python
(current_value - minimum ) / (maximum - minimum)
```

For example, if the current_value is 50, then the percentage of steps is 50%.

To set the current step value, you use the `setValue()` method. To get the current value, you use the `value()` method. To reset the progress bar so that it shows no progress, you use the reset() method.

You can use the `setMinimum() and setMaximum()` methods to change the default min, max value.


## 1 Simple example

In below code, we use two button to set the progress in the progress bar



In [1]:
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QPushButton, QVBoxLayout, QHBoxLayout, QProgressBar
from PyQt6.QtCore import Qt


class MainWindow(QWidget):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.setGeometry(100, 100, 300, 50)
        self.setWindowTitle('QProgressBar Demo')

        # first layer layout
        layout = QVBoxLayout()
        self.setLayout(layout)

        # sub layout of the 1st layer
        # contains the progress bar
        upSec = QHBoxLayout()
        self.progress_bar = QProgressBar(self)
        upSec.addWidget(self.progress_bar)

        layout.addLayout(upSec)

        # sub layout contains the two buttons
        downSec = QHBoxLayout()
        self.btn_progress = QPushButton('Progress', clicked=self.progress)
        self.bnt_reset = QPushButton('Reset', clicked=self.reset)

        # align buttons center
        # The addStretch method adds a QSpacerItem to the end of a box layout.
        # A QSpacerItem is an adjustable blank space.
        # Using vbox.addStretch(1) will add a zero-width spacer-item that expands
        # vertically from the top of the layout downwards.
        # Using hbox.addStretch(1) will add a zero-height spacer-item that expands
        # horizontally from the left of the layout rightwards.
        downSec.addStretch()
        downSec.addWidget(self.btn_progress)
        downSec.addWidget(self.bnt_reset)
        downSec.addStretch()

        layout.addLayout(downSec)
        self.current_value = 0
        self.show()

    def reset(self):
        self.current_value = 0
        self.progress_bar.reset()

    def progress(self):
        if self.current_value  <= self.progress_bar.maximum():
            self.current_value += 5
            self.progress_bar.setValue(self.current_value)



app = QApplication(sys.argv)
window = MainWindow()
sys.exit(app.exec())


qt.qpa.xcb: X server does not support XInput 2


SystemExit: 0

/home/pengfei/.cache/pypoetry/virtualenvs/python-desktop-app-framework-xY3vli_t-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 2 Another example

In below example, the `start` button will start a loop which update the progress automatically with a delay timer. As this time the `startProgres` function will take some time to finish, so the `GUI is frozen and unresponsive` until the counter meets the TIME_LIMIT condition. Try to click on `start` many times, you will notice it will make the function `startProgress` run multiple times. It means that even the GUI is frozen, the user action is registered in the event loop.

The example 1 does not have this behavior, because the action is finished instantly after the button is clicked. So users can not feel the frozen, but it freezes too.

In [ ]:
import sys
import time

from PyQt6.QtWidgets import (QApplication, QProgressBar, QPushButton, QWidget, QVBoxLayout, QLabel)

TIME_LIMIT = 100


class MainWindow(QWidget):
    """
    Simple widget that consists of a Progress Bar and a Button.
    Clicking on the button results in the start of a timer and
    updates the progress bar.
    """

    def __init__(self):
        super().__init__()
        self.count=0
        self.setWindowTitle('Progress Bar')
        self.setGeometry(100, 100, 300, 125)
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.initUI()
        self.show()

    def initUI(self):
        self.countLabel = QLabel()
        self.countLabel.setText(f"Current Count: {self.count}")
        self.countBtn = QPushButton("Add 1")
        self.countBtn.clicked.connect(self._updateCount)
        # set up progressbar
        self.progress = QProgressBar(self)
        self.progress.setMaximum(100)

        # setup button to trigger the progress
        self.button = QPushButton('Start', self)

        # add them to layout
        self.layout.addWidget(self.countLabel)
        self.layout.addWidget(self.countBtn)
        self.layout.addWidget(self.progress)
        self.layout.addWidget(self.button)

        self.button.clicked.connect(self.startProgress)

    def startProgress(self):
        """
        This method update 1% in the progress bar after every 0.1 second
        Returns
        -------

        """
        count = 0
        while count < TIME_LIMIT:
            count += 1
            time.sleep(0.1)
            self.progress.setValue(count)

    def _updateCount(self):
        self.count += 1
        self.countLabel.setText(f"Current Count: {self.count}")


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    sys.exit(app.exec())

## 3. Use QThread

We can use QThread to make the `GUI responsive`.


### 3.1 First example

In below code, we implement a new class called `ProgresBarUpdater`. This class extends QThread, it uses an internal counter to update the progress bar. The update action is done by emitting a signal with a parameter count. The slot which receives the signal can use this parameter to reset the progress bar.

It will be run as a new thread outside the main thread. So The GUI will remain responsive during the execution of the new thread.

In [ ]:
import sys
import time

from PyQt6.QtCore import QThread, pyqtSignal
from PyQt6.QtWidgets import (QApplication, QProgressBar, QPushButton, QWidget, QVBoxLayout, QLabel)

TIME_LIMIT = 100


class ProgresBarUpdater(QThread):
    """
    This class extends QThread, it uses an internal counter to update the progress bar.
    The update action is done by emitting a signal with a parameter count. The slot which receives
    the signal can use this parameter to reset the progress bar.
    """
    countChanged = pyqtSignal(int)

    def run(self):
        timeInterval = 0.1
        count = 0
        while count < TIME_LIMIT:
            count += 1
            time.sleep(timeInterval)
            # send a signal with a parameter count
            self.countChanged.emit(count)


class MainWindow(QWidget):
    """
    Simple dialog that consists of a Progress Bar and a Button.
    Clicking on the button results in the start of a timer and
    updates the progress bar.
    """

    def __init__(self):
        super().__init__()
        self.count = 0
        self.setWindowTitle('Threading Progress Bar')
        self.setGeometry(100, 100, 350, 125)
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.initUI()
        self.show()

    def initUI(self):
        self.countLabel = QLabel()
        self.countLabel.setText(f"Current Count: {self.count}")
        self.countBtn = QPushButton("Add 1")
        self.countBtn.clicked.connect(self._updateCount)

        self.progress = QProgressBar(self)
        self.progress.setMaximum(100)

        self.startBtn = QPushButton('Start', self)
        self.startBtn.clicked.connect(self._startProgress)

        self.layout.addWidget(self.countLabel)
        self.layout.addWidget(self.countBtn)
        self.layout.addWidget(self.progress)
        self.layout.addWidget(self.startBtn)

    def _startProgress(self):
        # creat an instance of ProgresBarUpdater which is a thread
        self.myThread = ProgresBarUpdater()
        # connect the signal send by process to slot
        self.myThread.countChanged.connect(self._onCountChanged)
        # start the thread
        self.myThread.start()

    def _updateCount(self):
        self.count += 1
        self.countLabel.setText(f"Current Count: {self.count}")

    def _onCountChanged(self, value):
        """
        This method (slot) update the progress bar value
        Parameters
        ----------
        value

        Returns
        -------

        """
        self.progress.setValue(value)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    sys.exit(app.exec())


### 3.2 A more complex example

A Thread can emit multiple signal, in below example, the thread will emit two signals, one to update the progress bar, second to write log on a custom scroll label.

In [ ]:
import sys
import time
from datetime import datetime

from PyQt6.QtCore import QThread, pyqtSignal, Qt
from PyQt6.QtWidgets import (QApplication, QProgressBar, QPushButton, QWidget, QVBoxLayout, QLabel, QScrollArea)

TIME_LIMIT = 100


class ScrollLabel(QScrollArea):

    # constructor
    def __init__(self, *args, **kwargs):
        QScrollArea.__init__(self, *args, **kwargs)

        # making widget resizable
        self.setWidgetResizable(True)

        # making qwidget object
        content = QWidget(self)
        self.setWidget(content)

        # vertical box layout
        lay = QVBoxLayout(content)

        # creating label
        self.label = QLabel(content)

        # setting alignment to the text
        self.label.setAlignment(Qt.AlignmentFlag.AlignLeft | Qt.AlignmentFlag.AlignTop)

        # making label multi-line
        self.label.setWordWrap(True)

        # adding label to the layout
        lay.addWidget(self.label)

    # the setText method
    def setText(self, text):
        # setting text to the label
        self.label.setText(text)

    def text(self):
        return self.label.text()

class ProgresBarUpdater(QThread):
    """
    This class extends QThread, it uses an internal counter to update the progress bar.
    The update action is done by emitting a signal with a parameter count. The slot which receives
    the signal can use this parameter to reset the progress bar.
    """
    countChanged = pyqtSignal(int)
    updateDate = pyqtSignal(str)

    def run(self):
        timeInterval = 0.1
        count = 0
        while count < TIME_LIMIT:
            count += 1
            time.sleep(timeInterval)
            # send a signal with a parameter count
            self.countChanged.emit(count)
            self.updateDate.emit(f"Process running, current time: {datetime.now()}")


class MainWindow(QWidget):
    """
    Simple dialog that consists of a Progress Bar and a Button.
    Clicking on the button results in the start of a timer and
    updates the progress bar.
    """

    def __init__(self):
        super().__init__()
        self.count = 0
        self.setWindowTitle('Threading Progress Bar')
        self.setGeometry(100, 100, 350, 125)
        self.layout = QVBoxLayout()
        self.setLayout(self.layout)
        self.initUI()
        self.show()

    def initUI(self):
        self.countLabel = QLabel()
        self.countLabel.setText(f"Current Count: {self.count}")
        self.countBtn = QPushButton("Add 1")
        self.countBtn.clicked.connect(self._updateCount)

        self.progress = QProgressBar(self)
        self.progress.setMaximum(100)

        self.logLabel = ScrollLabel()

        self.startBtn = QPushButton('Start', self)
        self.startBtn.clicked.connect(self._startProgress)

        self.layout.addWidget(self.countLabel)
        self.layout.addWidget(self.countBtn)
        self.layout.addWidget(self.progress)
        self.layout.addWidget(self.logLabel)
        self.layout.addWidget(self.startBtn)

    def _startProgress(self):
        # creat an instance of ProgresBarUpdater which is a thread
        self.myThread = ProgresBarUpdater()
        # connect the countChanged signal to slot _onCountChanged
        self.myThread.countChanged.connect(self._onCountChanged)

        # connect
        self.myThread.updateDate.connect(self._onUpdateDate)
        # start the thread
        self.myThread.start()

    def _updateCount(self):
        self.count += 1
        self.countLabel.setText(f"Current Count: {self.count}")

    def _onCountChanged(self, value):
        """
        This method (slot) update the progress bar value
        Parameters
        ----------
        value

        Returns
        -------

        """
        self.progress.setValue(value)

    def _onUpdateDate(self, msg: str):
        oldText = self.logLabel.text()
        self.logLabel.setText("<br>".join([oldText, msg]))


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    sys.exit(app.exec())
